<br>

# Introdução

In [ ]:
#!pip3 install dask --upgrade
#!pip3 install dask-labextension  --upgrade
#!pip3 install pyarrow --upgrade        # Necessário para usar o parquet
#!pip3 install traquitanas --upgrade
#!jupyter labextension install dask-labextension

In [1]:
import os
import sys
import time
import ctypes
import datetime
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

In [2]:
import dask.dataframe as dd
from dask import compute
from dask.delayed import delayed
from dask.distributed import Client, LocalCluster
from dask.distributed import wait, progress
from dask.diagnostics import ProgressBar

In [3]:
import dask
import dask.distributed  # populate config with distributed defaults
dask.config.get('distributed.client')

{'heartbeat': '5s',
 'scheduler-info-interval': '2s',
 'security-loader': None,
 'preload': [],
 'preload-argv': []}

In [4]:
mod_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.append(mod_path)
from sisagua.ibge import *

In [5]:
from paths import *

<br>

## Client

In [6]:
import multiprocessing as mp
from dask.distributed import Client, LocalCluster

In [7]:
#import close_process
#close_process.process()

In [11]:
cluster = LocalCluster(
    n_workers=int(0.9 * mp.cpu_count()),
    threads_per_worker=4,
    memory_limit='4GB',
    processes=True,
    env={'MALLOC_TRIM_THRESHOLD_': '65536'}
)

client = Client(cluster)
client

/home/michel/Documents/conda/envs/pablocarreira-py39/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43549 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43549/status,
Dashboard: http://127.0.0.1:43549/status,Workers: 3
Total threads: 12,Total memory: 11.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43891,Workers: 3
Dashboard: http://127.0.0.1:43549/status,Total threads: 12
Started: Just now,Total memory: 11.18 GiB
Comm: tcp://127.0.0.1:45975,Total threads: 4
Dashboard: http://127.0.0.1:38339/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:32979,


<br>

## Parameters

In [12]:
# Parameters
cod_ibge = '3505203' # Bariri
#cod_ibge = '3501608' # Americana
#cod_ibge = '3548906' # São Carlos
#cod_ibge = '3526902' # Limeira
cod_ibge = '3550308' # São Paulo

In [13]:
estado_d = find_states(cod_ibge)
estado = estado_d['sigla']

cod_ibge_ajustado = adjust_id_ibge(cod_ibge)

Padrão IBGE novo, com código de 7 dígitos.
Correções necessárias aplicadas!
Código IBGE: 355030


<br>

## Dask Zifiles

In [14]:
# Parameters
filenames = [
    # Cadastro
    os.path.join(input_path_parquet_partitioned, 'cadastro', 'cadastro_pontos_captacao'),
    os.path.join(input_path_parquet_partitioned, 'cadastro', 'cadastro_populacao_abastecida'),
    os.path.join(input_path_parquet_partitioned, 'cadastro', 'cadastro_tratamento_de_agua'),
    
    # Controle
    os.path.join(input_path_parquet_partitioned, 'controle', 'controle_mensal_amostras_fora_padrao'),
    os.path.join(input_path_parquet_partitioned, 'controle', 'controle_mensal_demais_parametros'),
    os.path.join(input_path_parquet_partitioned, 'controle', 'controle_mensal_infraestrutura_operacionais'),
    os.path.join(input_path_parquet_partitioned, 'controle', 'controle_mensal_parametros_basicos_*'),
    os.path.join(input_path_parquet_partitioned, 'controle', 'controle_semestral_*'),
    
    # Vigilância
    os.path.join(input_path_parquet_partitioned, 'vigilancia', 'vigilancia_cianobacterias_cianotoxinas'),
    os.path.join(input_path_parquet_partitioned, 'vigilancia', 'vigilancia_demais_parametros'),
    os.path.join(input_path_parquet_partitioned, 'vigilancia', 'vigilancia_parametros_basicos_*'),
]

In [15]:
for filename in filenames:    
    # Path
    basename = os.path.basename(filename).replace('_*', '')
    subdir = os.path.basename(os.path.dirname(filename))
    print('Na pasta "{}", processando arquivo "{}"'.format(subdir, basename))
    
    # Set and create output paths
    output_path_city = os.path.join(
        output_path,
        '{}'.format(cod_ibge),
        'dados brutos',
        subdir,
    )
    os.makedirs(output_path_city, exist_ok=True)
    
    # Add Filter to Filename
    filter_path = os.path.join('Uf={}'.format(estado), 'Código Ibge={}/*.parquet'.format(cod_ibge_ajustado))
    filename = os.path.join(filename, filter_path)
    
    # Set Filters
    filters = [[('Uf', '==', estado), ('Código Ibge', '=', int(cod_ibge_ajustado))]]

    # Read Dataframes
    df = dd.read_parquet(
        filename,
        filters=filters,
    )

    # Calculate
    df = df.compute()
    df.head()

    # Write to Excel
    df.to_excel(
        os.path.join(output_path_city, '{}.xlsx'.format(basename)),
        index=False,
    )

Na pasta "cadastro", processando arquivo "cadastro_pontos_captacao"
Na pasta "cadastro", processando arquivo "cadastro_populacao_abastecida"
Na pasta "cadastro", processando arquivo "cadastro_tratamento_de_agua"
Na pasta "controle", processando arquivo "controle_mensal_amostras_fora_padrao"
Na pasta "controle", processando arquivo "controle_mensal_demais_parametros"
Na pasta "controle", processando arquivo "controle_mensal_infraestrutura_operacionais"
Na pasta "controle", processando arquivo "controle_mensal_parametros_basicos"


ValueError: This sheet is too large! Your sheet size is: 1873690, 22 Max sheet size is: 1048576, 16384

<br>

## End

In [ ]:
cluster.close()
time.sleep(1)
client.close()